## Importing libraries

In [5]:
import pandas as pd # to store data efficiently
import numpy as np # to handle numerical operations
import random # to generate random numbers
import os # to handle operations regarding file system
import wget # to download locally images
from ultralytics import YOLO # to import YOLO model
from tqdm import tqdm # to visualize loop progressions
import matplotlib.pyplot as plt # to create visualizations
import cv2 # to handle images
import re # to identify regu
import json


In [30]:
model = YOLO("../models/YOLO/yolo11n.pt")

In [ ]:
import cv2
import torch
import json


# Open the video
cap = cv2.VideoCapture('output.mp4')

# Get the width and height of the frames to set the output size
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the codec and create a VideoWriter object to save the output video
out = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 30, (frame_width, frame_height))

# Initialize the list for JSON data
detection_list = []

frame_number = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_list = []
    
    # Perform YOLO prediction
    res = model(frame, verbose=False)[0]
    class_mask = res.boxes.cls == 0  # Filter for class 0 (e.g., 'person')

    confidences = res.boxes.conf[class_mask]
    dimensions = res.boxes.xywhn[class_mask]

    # Draw the bounding boxes on the frame
    for index, (conf, dim) in enumerate(zip(confidences, dimensions)):
        x_center, y_center, width, height = dim.tolist()

        # Convert normalized coordinates to pixel coordinates
        x1 = int((x_center - width / 2) * frame_width)
        y1 = int((y_center - height / 2) * frame_height)
        x2 = int((x_center + width / 2) * frame_width)
        y2 = int((y_center + height / 2) * frame_height)

        # Draw the bounding box and label on the frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f'Person {conf:.2f}'
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Save detection info to JSON
        detection_dict = {
            'Img_ID': frame_number,
            'Confidence': conf.item(),
            'Dimensions': [x1, y1, x2, y2],
            'ID': f'{frame_number}{index}',
        }
        frame_list.append(detection_dict)
    
    # If no detections, still save the empty frame information
    if len(confidences) == 0:
        detection_dict = {
            'Img_ID': frame_number,
            'Confidence': 0,
            'Dimensions': [],
            'ID': frame_number,
        }
        frame_list.append(detection_dict)
    detection_list.append(frame_list)

    # Write the frame with bounding boxes to the output video
    out.write(frame)

    frame_number += 1

# Save the JSON data to a file
with open('predictions.json', 'w') as f:
    json.dump(detection_list, f, indent=4)

# Release the video capture and writer
cap.release()
out.release()
cv2.destroyAllWindows()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
